In [23]:
import pandas as pd
import os
import numpy as np

In [24]:
original_labelmap = {
    "T1":0,
    "T1_c":1,
    "T2":2,
    "T2-FLAIR":3,
    "FS" : 4,
    "PD":5,
    "SWI":6,
    "GRE":7,
    "T2*":8,
    "DWI":9,
    "ADC":10,
    "BOLD":11,
    "FlowSensitve":12,
    "PWI":13,
    "ASL":14,
    "Hippo":15,
    "DTI":16,
    "Other":17
}

In [25]:
all_binnings_dict = {
    'DDS_binning': {
        'T1_binned' : ['T1', 'Hippo'],
        'non_DDS': ['SWI','GRE','T2*','ADC','BOLD','FlowSensitve','ASL','DTI']
    },
    'alternative binning': {
        'T1_binned' : ['T1', 'Hippo'],
        'diffusion' : ['DWI', 'ADC', 'DTI'],
        'perfusion' : ['BOLD', 'ASL', 'FlowSensitve', 'PWI'],
        'suscept' : ['SWI','GRE','T2*']
    }
}

In [26]:
def createBinning(original_labelmap, binningDict):
    orignal_labelmap_copy = original_labelmap.copy()
    new_labelmap = {}
    transfer_array = np.zeros(len(original_labelmap))
    class_counter = 0
    for bin_name, bin_items in binningDict.items():
        new_labelmap[bin_name] = class_counter
        for item in list(bin_items):
            transfer_array[original_labelmap[item]] = class_counter
            orignal_labelmap_copy.pop(item)
        class_counter += 1
    for name, item in orignal_labelmap_copy.items():
        new_labelmap[name] = class_counter
        transfer_array[item] = class_counter
        class_counter += 1
    unchangedLabels = orignal_labelmap_copy
    return new_labelmap, transfer_array, class_counter, unchangedLabels
    

In [27]:
createBinning(original_labelmap, all_binnings_dict['DDS_binning'])

IndexError: index 17 is out of bounds for axis 0 with size 17

In [ ]:
def apply_binning_to_df(original_df, transfer_array, suffix=""):
    df = original_df.copy()
    suffix = '_'+suffix
    df[f'label{suffix}'] = df['label'].apply(lambda x: transfer_array[x])
    return df